# Streamline Modeling With Amazon SageMaker Studio and Amazon Experiments SDK

Modeling phase is a highly iterative process in Machine Learning projects, where Data Scientists experiment with various data pre-processing and feature engineering strategies, intertwined with different model architectures, which are then trained with disparate sets of hyperparameter values. This highly iterative process, with many moving parts, can, over time, manifest into a tremendous headache in terms of keeping track of all design decisions applied in each iteration and how the training and evaluation metrics of each iteration compare to the previous versions of the model.

This notebook walks you through an end-to-end example of how [Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-studio.html) can effectively leverage [Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) to organize, track, visualize and compare our iterative work during development of a Keras model, trained to predict the age of an abalone, a sea snail, based on a set of features that describe it. While this example is specific to Keras, the same approach can be extended to other Machine Learning frameworks and algorithms. 

Amazon SageMaker Studio and Amazon SageMaker Experiments were unveiled at the [AWS Re:invent](https://aws.amazon.com/new/reinvent/), at the end of 2019:
* [SageMaker Studio Announcement](https://aws.amazon.com/about-aws/whats-new/2019/12/introducing-amazon-sagemaker-studio-the-first-integrated-development-environment-ide-for-machine-learning/?trk=ls_card)
* [SageMaker Experiments Announcement](https://aws.amazon.com/about-aws/whats-new/2019/12/introducing-amazon-sagemaker-experiments-organize-track-and-compare-your-machine-learning-training-experiments-on-amazon-sagemaker/?trk=ls_card)

In this walkthrough, we will explore how Amazon SageMaker Studio, and the [Experiments SDK](https://sagemaker-experiments.readthedocs.io/en/latest/), which has been [open-sourced](https://github.com/aws/sagemaker-experiments), can be utilized to experiment with a Keras model and track data preprocessing required to prepare data for the model's consumption.

Now, before we dive into hands-on exercise, let's first take a step back and discuss the building blocks of each Experiment and their referential relationships.
* **Experiment** - a Machine Learning problem that we want to solve. Each experiment consists of a collection of Trials. Note that the name of an experiment must be unique in a given region of a particular AWS account.
* **Trial** - an execution of a data-science workflow related to an experiment. Each Trial consists of several Trial Components. Note that the name of a trial must be unique in a given region of a particular AWS account.
* **Trial Component** - a stage in a given trial. For instance, as we will see in our example, we will create one Trial Component for data pre-preprocessing stage and one Trial Component for model training. Similarly, in other use cases, we can also have a Trial Component for data post-processing. Unlike Experiments and Trials, Trial Components do not have to be uniquely named as they tend to represent the typical and very common stages in an ML pipeline.
* **Tracker** - a mechanism that records various metadata about a particular Trial Component, including any Parameters, Inputs, Outputs, Artifacts and Metrics. When creating a Tracker, each Tracker is linked to a particular Training Component.

![](img/experiment_structure_t.png)

## Environment Setup

Now that we've set a rock-solid foundation on the key building blocks of Experiments SDK, let's dive into the fun hands-on component of this exercise. Before we jump into data pre-processing and feature engineering, let's first set up our environment and make sure that we have all the packages imported and variables defined.

First, select `Python 3 (TensorFlow 2 CPU Optimized)` kernel for your notebook in your Studio environment. The smallest instance with 2 virtual CPUs and 4 GiB of memory will be more than sufficient for our exercise.

Let's begin by installing `sagemaker-experiments` package, which will enable us to work with Experiments SDK. Alongside, let's also istall `s3fs` package, to enable our pandas dataframes to easily work with objects in S3.

In [1]:
! pip uninstall -y awscli boto3 s3fs sagemaker-experiments # clean up dependencies
! pip install awscli==1.18.152 s3fs==0.5.0 sagemaker-experiments==0.1.24 # install packages required for the demo

Found existing installation: awscli 1.18.22
Uninstalling awscli-1.18.22:
  Successfully uninstalled awscli-1.18.22
Found existing installation: boto3 1.12.22
Uninstalling boto3-1.12.22:
  Successfully uninstalled boto3-1.12.22
Found existing installation: sagemaker-experiments 0.1.7
Uninstalling sagemaker-experiments-0.1.7:
  Successfully uninstalled sagemaker-experiments-0.1.7
  Using cached awscli-1.18.152-py2.py3-none-any.whl (3.4 MB)
  Using cached s3fs-0.5.0-py3-none-any.whl (21 kB)
  Using cached sagemaker_experiments-0.1.24-py3-none-any.whl (36 kB)
  Using cached botocore-1.18.11-py2.py3-none-any.whl (6.7 MB)
  Using cached fsspec-0.8.3-py3-none-any.whl (88 kB)
  Using cached aiobotocore-1.1.1-py3-none-any.whl (45 kB)
Processing ./.cache/pip/wheels/84/37/2b/cf289b3d2fbebbc8a8ecafdfcc526222a9c9d1feabb3d1447c/boto3-1.15.11-py2.py3-none-any.whl
  Using cached aiohttp-3.6.2-cp36-cp36m-manylinux1_x86_64.whl (1.2 MB)
  Using cached aioitertools-0.7.0-py3-none-any.whl (20 kB)
  Using c

In [2]:
import os
import sagemaker as sm
import boto3
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing as pp
import pickle
from sagemaker.tensorflow import TensorFlow
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

role = sm.get_execution_role()
sm_bucket = sm.session.Session().default_bucket()
project_path = 'experiments-keras-abalone'
source_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/'
processed_data_path = os.path.join('s3://',sm_bucket,project_path,'data/processed/')
artifacts_path = os.path.join(project_path,'artifacts')
output_path = os.path.join('s3://',sm_bucket,project_path,'output/')

## Data Pre-processing and Feature Engineering

Excellent! Now, let's dive into data pre-processing and feature engineering. First, let's pull our dataset, and its metadata, from [the archive of UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/) and familiarize with the features by reviewing the portion of the metadata file that describes them.

In [3]:
urlretrieve(source_url+'abalone.data', 'abalone.data')
urlretrieve(source_url+'abalone.names', 'abalone.names')
! cat abalone.names | grep 'Attribute information:' -A 16

7. Attribute information:

   Given is the attribute name, attribute type, the measurement unit and a
   brief description.  The number of rings is the value to predict: either
   as a continuous value or as a classification problem.

	Name		Data Type	Meas.	Description
	----		---------	-----	-----------
	Sex		nominal			M, F, and I (infant)
	Length		continuous	mm	Longest shell measurement
	Diameter	continuous	mm	perpendicular to length
	Height		continuous	mm	with meat in shell
	Whole weight	continuous	grams	whole abalone
	Shucked weight	continuous	grams	weight of meat
	Viscera weight	continuous	grams	gut weight (after bleeding)
	Shell weight	continuous	grams	after being dried
	Rings		integer			+1.5 gives the age in years


Now that we understand the features, let's see how many examples we have and print a few examples to see the data.

In [4]:
col = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight','Rings']
df = pd.read_csv('abalone.data',header=None, names=col)
print('The dataset contains {} examples with {} features and the label in the last column.\n'.format(df.shape[0],df.shape[1]))
df.head()

The dataset contains 4177 examples with 9 features and the label in the last column.



,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Let's move on to data transformations and feature engineering, starting with nominal features. We will apply one-hot encoding with a dropout to the nominal feature and drop the original one. We will then rename the encoded columns to ensure that their names clearly illustrate the data they contain.

In [5]:
cat_features = [df.columns[0]]
encoders = {}

for feature in cat_features:
    oe = pp.OneHotEncoder(drop='first')
    oe.fit(df[[feature]])
    encoded_feature = pd.DataFrame(oe.transform(df[[feature]]).toarray())
    df = encoded_feature.join(df)
    encoders[feature] = oe

df.drop(columns=cat_features,inplace=True)
df.rename(columns={0:'gender_oh1', 1:'gender_oh2'},inplace=True)
df.head()

,gender_oh1,gender_oh2,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0.0,1.0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0.0,1.0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.0,0.0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0.0,1.0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,1.0,0.0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Next, we will tackle continuous numerical features and apply MinMaxScaler on them to confine all values between 0 and 1.

In [6]:
num_features = df.columns[2:-1]
scalers = {}

for feature in num_features:
    mms = pp.MinMaxScaler()
    mms.fit(df[[feature]])
    scaled_feature = pd.DataFrame(mms.transform(df[[feature]]))
    df[feature] = scaled_feature
    scalers[feature] = mms

df.head()

,gender_oh1,gender_oh2,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0.0,1.0,0.513514,0.521008,0.084071,0.181335,0.150303,0.132324,0.147982,15
1,0.0,1.0,0.371622,0.352941,0.079646,0.079157,0.066241,0.063199,0.068261,7
2,0.0,0.0,0.614865,0.613445,0.119469,0.239065,0.171822,0.185648,0.207773,9
3,0.0,1.0,0.493243,0.521008,0.110619,0.182044,0.144250,0.149440,0.152965,10
4,1.0,0.0,0.344595,0.336134,0.070796,0.071897,0.059516,0.051350,0.053313,7


We can now proceed with splitting the pre-processed dataset for training and testing, where we will retain 80% of the dataset for training and use the remaining examples as a hold-out validation set. We will also further split the labels from the features since our Keras model, which we will review shortly, expects features and labels to be fed separately.

In [7]:
# Split the dataset into train, test and validation
train_perc = 80
train_cnt = int(np.floor(df.shape[0]*train_perc/100))

train_df = df[:train_cnt]
test_df = df[train_cnt:]

train_features = train_df.drop(columns=['Rings'])
train_labels = train_df['Rings']
test_features = test_df.drop(columns=['Rings'])
test_labels = test_df['Rings']

print('Train record count: {}\nTest record count: {}'.format(train_df.shape[0], test_df.shape[0]))

Train record count: 3341
Test record count: 836


Finally, let's proceed with pushing the split datasets to the processed path in S3, from where the model can import them during the training. We will also combine our fitted encoders and scalers into a dictionary and serialize it as a pickle file. As we will see shortly, we will attach these fitted processors to our Trial Component that tracks data pre-processing stage.

In [8]:
train_features.to_csv(os.path.join(processed_data_path,'abalone_train_features.csv'),header=False, index=False)
train_labels.to_csv(os.path.join(processed_data_path,'abalone_train_labels.csv'),header=False, index=False)
test_features.to_csv(os.path.join(processed_data_path,'abalone_test_features.csv'),header=False, index=False)
test_labels.to_csv(os.path.join(processed_data_path,'abalone_test_labels.csv'),header=False, index=False)

with open('preprocessors.pickle','wb') as obj:
    pickle.dump({'encoders' : encoders, 'scalers': scalers}, obj)

Now, let's utilize the Experiments SDK to create a Pre-processing Trial Component and keep track of what've accomplished so far. We start by creating an experiment. Remember that each experiment name must be unique within a given region of a particular AWS account.

Once we have our experiment instantiated, we will proceed with creating a Tracker to capture various details about the data pre-processing we completed so far.

In [9]:
sm = boto3.client('sagemaker')
ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')

abalone_experiment = Experiment.create(
    experiment_name = 'predict-abalone-age-' + ts,
    description = 'Predicting the age of an abalone based on a set of features describing it',
    sagemaker_boto_client=sm)


with Tracker.create(display_name='Pre-processing', sagemaker_boto_client=sm, artifact_bucket=sm_bucket, artifact_prefix=artifacts_path) as tracker:
    tracker.log_parameters({
        'train_test_split': 0.8
    })
    tracker.log_input(name='raw data', media_type='s3/uri', value=source_url)
    tracker.log_output(name='preprocessed data', media_type='s3/uri', value=processed_data_path)
    tracker.log_artifact(name='preprocessors', media_type='s3/uri', file_path='preprocessors.pickle')
    
processing_component = tracker.trial_component

Fantastic! We now have our experiment ready and we've already done our due diligence to capture our data pre-processing approach as a Trial Component. Next, let's dive into the modeling phase.

## Modeling with SageMaker Experiments

So far, we've done solid amount of work to preprocess data for our model. Before we proceed with training the model, let's take a moment to familiarize with it. We observe that our model has two fully connected hidden layers with a variable number of neurons and variable activation functions. This flexibility will enable us to easily pass these values as arguments to a training job and quickly parallelize our experimentation with several model architectures.

We have Mean Squared Logarithmic Error defined as the loss function and the model will be using Adam optimization algorithm. Finally, the model will track Mean Squared Logarithmic Error as our metric, which will automatically propagate into our Training Trial Component in our Experiment, as we will see shortly.

In [10]:
# Review the architecture of the model
!pygmentize 'entrypoint.py'

''' Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
    SPDX-License-Identifier: MIT-0 '''

# Seed random number generators to ensure identical results for everyone
from numpy.random import seed
seed(42)
from tensorflow import random
random.set_seed(42)
########################################################################

import argparse
import os
import numpy as np
import json
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


def model(x_train, y_train, x_test, y_test, args):
    """Generate a simple model"""
    model = Sequential([
                Dense(args.l1_size, activation=args.l1_activation, kernel_initializer='normal'),
                Dense(args.l2_size, activation=args.l2_activation, kernel_initializer='normal'),
                Dense(1, activation='linear')
    ])

    model.compile(optimizer=Adam(learning_rate=args.learning_rate),
              

We are ready to define a few sets of hyperparameters that we want to experiment with and kick off our training jobs to run in parallel. Note that we can also easily capture custom metrics during training by configuring a regex pattern to match the desired values in the logs generated by the training job. In our case, for demonstration purposes, we will capture the number of samples (`num_train_samples`) that the Keras framework reports as being available for training the model.

Note that, in our example, we do not make any changes to our pre-processing strategy between different trials, so we will attach the same Pre-processing Trial Component that we create earlier, to each of the Trials we create in the next step.

**Note:** If you elect to use your own execution role instead of the one defined in this notebook, please make sure that the access policy attached to your custom execution role, that you pass to your training job, allows `sagemaker:BatchGetMetrics` and `sagemaker:BatchPutMetrics` actions, in order for us to be able to capture non-custom metrics during training and visualize them in Studio. The four training jobs that we will kick off next will take only about 5 minutes to complete.

In [11]:
# Define sets of hyperparameters, create a trial for each and kick of training jobs
hyperparameters_groups=[{
                         'learning_rate': 0.001,
                         'epochs': 12,
                         'batch_size': 128,
                         'l1_size': 20,
                         'l1_activation': 'tanh',
                         'l2_size': 20,
                         'l2_activation': 'relu'
                        },
                        {
                         'learning_rate': 0.001,
                         'epochs': 12,
                         'batch_size': 128,
                         'l1_size': 10,
                         'l1_activation': 'tanh',
                         'l2_size': 10,
                         'l2_activation': 'relu'
                        },
                        {
                         'learning_rate': 0.001,
                         'epochs': 12,
                         'batch_size': 128,
                         'l1_size': 20,
                         'l1_activation': 'relu',
                         'l2_size': 10,
                         'l2_activation': 'relu'
                        },
                        {
                         'learning_rate': 0.001,
                         'epochs': 12,
                         'batch_size': 128,
                         'l1_size': 10,
                         'l1_activation': 'relu',
                         'l1_size': 5,
                         'l1_activation': 'relu'
                        }]


for i,hp_set in enumerate(hyperparameters_groups):

    ts = datetime.now().strftime('%Y-%m-%d-%H-%M-%S-%f')
    abalone_trial = abalone_experiment.create_trial(trial_name='abalone-trial-' + str(i) + '-'+ ts)
    abalone_trial.add_trial_component(processing_component)
    
    abalone_estimator = TensorFlow(entry_point='entrypoint.py',
                                   role=role,
                                   framework_version='2.1.0',
                                   py_version='py3',
                                   hyperparameters=hp_set,
                                   train_instance_count=1,
                                   train_instance_type='ml.m5.large',
                                   output_path = output_path,
                                   metric_definitions=[
                                       {'Name': 'num_train_samples', 'Regex': 'Train on (\d+) samples'}
                                   ]
                                   )

    job_name = 'abalone-train-' + str(i) + '-'+ ts
    
    abalone_estimator.fit(processed_data_path,
                          job_name=job_name,
                          wait=False,
                          experiment_config={
                                            'ExperimentName': abalone_experiment.experiment_name,
                                            'TrialName': abalone_trial.trial_name,
                                            'TrialComponentDisplayName': 'Training',
                                            })

INFO:sagemaker:Creating training-job with name: abalone-train-0-2020-10-03-18-49-53-209367
INFO:sagemaker:Creating training-job with name: abalone-train-1-2020-10-03-18-49-53-691462
INFO:sagemaker:Creating training-job with name: abalone-train-2-2020-10-03-18-49-56-379229
INFO:sagemaker:Creating training-job with name: abalone-train-3-2020-10-03-18-50-01-098410


## Exploration

Upon completion of the training jobs, we can, in just a few seconds, start visualizing how different variations of the model compare in terms of the metrics collected during model training. For instance, in just a few clicks, we can visualize how the loss has been decreasing by epoch for each variation of the model and very quickly observe the model architecture that is most effective in decreasing the loss.

![](img/visualize_loss.png)

You can create the same plot by following the five simple steps listed below:
1. Select the *SageMaker Experiments List* icon on the left sidebar.
2. Double-click on your experiment to open it and use Shift key on your keyboard to select all four trials.
3. Right-click on any of the highlighted trials and select *Open in trial component list*.
4. Use Shift key on your keyboard to select the four Trial Components representing the Training jobs and click on *Add chart* button.
5. Click on *New chart* and customize it to plot the collected metrics that you would like to analyze.

![](img/visualization_steps_line.gif)

Similarly, we can generate a scatter plot that helps us determine whether there is a relationship between the size of the first hidden layer in the network and the Mean Squared Logarithmic Error.

![](img/scatter_msle.png)


Wow! How quick and effortless was that?! I encourage you to further explore plotting various other metrics on your own.

Next, let's select our best performing trial (`abalone-trial-0`) and double-click on it. As expected, we see two Trial Components. One captures our data Pre-processing work, and the other reflects our Model Training.

When we open the Training Trial Component, we can see that it captures all the hyper parameters, the input to the model, where the trained model was placed in S3, etc. Similarly, when we open the *Preprocessing* component, we see that it captures where the source data came from, where the processed data was stored in S3, as well as, where we can easily find our trained encoder and scalers, which we've packaged into the `preprocessors.pickle` artifact.

![](img/review_artifacts.gif)



## Cleanup

What a fun exploration this has been! Let's now clean up after ourselves by hierarchically deleting all elements of the experiment that we created in this notebook. The function call in the last cell has been commented out to ensure that you do not accidentally delete your experiment before getting a chance to explore it, in case if you execute the entire notebook at once (*Run All Cells* option). To delete your experiment, uncomment the last cell and execute it.

I hope that you enjoyed diving into the intricacies of Experiments SDK and exploring how SageMaker Studio smoothly integrates with it, enabling you to lose yourself in experimentation with your Machine Learning model without losing any track of the hard work you've done!

In [12]:
# abalone_experiment.delete_all('--force')